# <center>PDL Lab15. Transfer Learning using CNN</center>


### 205229103

### Arzoo Sah

### 1.import neccesary modules 


In [1]:
import datetime
import keras
import pandas as pd
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

### 2. Initialize some parameters

In [2]:
now = datetime.datetime.now            #get current time

batch_size = 128
num_classes = 5
epochs = 5

img_rows, img_cols = 28, 28
filters = 32
pool_size = 2
kernel_size = 3

### 3. Partition MNIST dataset

In [3]:
# the data, shuffled and split between train and test sets

(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [4]:
x_train_lt5 = x_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]

x_test_lt5 = x_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

x_train_gte5 = x_train[y_train >= 5]
y_train_gte5 = y_train[y_train >= 5]-5

x_test_gte5 = x_test[y_test >= 5]
y_test_gte5 = y_test[y_test >= 5]-5

### 4. Define the 'feature' layers

In [5]:
INPUT_SHAPE = (img_rows,img_cols,1)

feature_layers=[Conv2D(filters=filters,kernel_size=kernel_size,activation='relu',input_shape=INPUT_SHAPE),
                Conv2D(filters=filters,kernel_size=kernel_size,activation='relu',input_shape=INPUT_SHAPE),
                MaxPooling2D(pool_size=pool_size),
                Dropout(0.25),
                Flatten()]

### 5. Define the "classification" layers

In [6]:
classification_layers = [Dense(128,activation='relu'),
                         Dropout(0.5),
                         Dense(5,activation='softmax')]

### 6. Define a Sequential model

In [7]:
model = Sequential(feature_layers+classification_layers)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 4608)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               589952    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0

### 7. Create a function 

In [8]:
def train_model(model,train,test,num_classes):
    
    train[0] = train[0].reshape(train[0].shape[0],img_rows,img_cols,1)
    test[0] = test[0].reshape(test[0].shape[0],img_rows,img_cols,1)

    train[0] = train[0].astype('float32')
    test[0] = test[0].astype('float32')
  
    train[0] /= 255
    test[0] /= 255

    print(train[0].shape)
    print(test[0].shape)

    s = pd.Series(train[1])
    train[1] = pd.get_dummies(s)
    train[1] = train[1].values

    s = pd.Series(test[1])
    test[1] = pd.get_dummies(s)
    test[1] = test[1].values


    model.compile(optimizer='adadelta',loss='categorical_crossentropy',metrics=['accuracy'])

    train_start_time=now()
 
    model.fit(train[0],train[1],epochs=epochs,verbose=2)
  
    print('\n')
    print('****************************************************************************************')
    print('\n')
  
    print('Training time: %s' % (now() - train_start_time))
  
    score=model.evaluate(test[0],test[1],verbose=0)
  
    print('test loss ',score[0])
    print('test accuracy ',score[1])

### 8. Train your model on the digits 5,6,7,8,9

In [9]:
train_model(model,[x_train_gte5,y_train_gte5],[x_test_gte5,y_test_gte5],num_classes)

(29404, 28, 28, 1)
(4861, 28, 28, 1)
Epoch 1/5
919/919 - 6s - loss: 1.5859 - accuracy: 0.2623
Epoch 2/5
919/919 - 4s - loss: 1.5261 - accuracy: 0.3862
Epoch 3/5
919/919 - 4s - loss: 1.4505 - accuracy: 0.5143
Epoch 4/5
919/919 - 4s - loss: 1.3485 - accuracy: 0.6159
Epoch 5/5
919/919 - 4s - loss: 1.2263 - accuracy: 0.6754


****************************************************************************************


Training time: 0:00:41.670260
test loss  1.097296118736267
test accuracy  0.8335733413696289


### 9. Freeze Feature Layers

In [10]:
for l in feature_layers:
 l.trainable = False

### 10. Print model summary()

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 4608)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               589952    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0

### 11. Now, let's train our model on the digits 0, 1, 2, 3, 4

In [12]:
train_model(model,[x_train_lt5, y_train_lt5],[x_test_lt5, y_test_lt5], num_classes)

(30596, 28, 28, 1)
(5139, 28, 28, 1)
Epoch 1/5
957/957 - 4s - loss: 1.4260 - accuracy: 0.4382
Epoch 2/5
957/957 - 3s - loss: 1.2617 - accuracy: 0.5486
Epoch 3/5
957/957 - 4s - loss: 1.1258 - accuracy: 0.6617
Epoch 4/5
957/957 - 4s - loss: 1.0064 - accuracy: 0.7506
Epoch 5/5
957/957 - 4s - loss: 0.9075 - accuracy: 0.8028


****************************************************************************************


Training time: 0:00:21.074518
test loss  0.787869930267334
test accuracy  0.8914185762405396


### 12. Now write code to reverse this training process

In [13]:
INPUT_SHAPE = (img_rows,img_cols,1)

feature_layers=[Conv2D(filters=filters,kernel_size=kernel_size,activation='relu',input_shape=INPUT_SHAPE),
                Conv2D(filters=filters,kernel_size=kernel_size,activation='relu',input_shape=INPUT_SHAPE),
                MaxPooling2D(pool_size=pool_size),
                Dropout(0.25),
                Flatten()]

classification_layers = [Dense(128,activation='relu'),
                         Dropout(0.5),
                         Dense(5,activation='softmax')]               

In [14]:
modelR = Sequential(feature_layers+classification_layers)
modelR.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 12, 12, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4608)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               589952    
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)              

In [15]:
train_model(modelR,[x_train_lt5,y_train_lt5],[x_test_lt5,y_test_lt5],num_classes)

(30596, 28, 28, 1)
(5139, 28, 28, 1)
Epoch 1/5
957/957 - 5s - loss: 1.5644 - accuracy: 0.3242
Epoch 2/5
957/957 - 4s - loss: 1.4418 - accuracy: 0.5800
Epoch 3/5
957/957 - 4s - loss: 1.2777 - accuracy: 0.7247
Epoch 4/5
957/957 - 4s - loss: 1.0661 - accuracy: 0.7970
Epoch 5/5
957/957 - 4s - loss: 0.8453 - accuracy: 0.8323


****************************************************************************************


Training time: 0:00:22.095582
test loss  0.638266921043396
test accuracy  0.9340338706970215


In [16]:
for l in feature_layers:
 l.trainable = False

In [17]:
modelR.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 12, 12, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4608)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               589952    
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)              

In [18]:
train_model(modelR,[x_train_gte5,y_train_gte5],[x_test_gte5,y_test_gte5],num_classes)

(29404, 28, 28, 1)
(4861, 28, 28, 1)
Epoch 1/5
919/919 - 4s - loss: 1.5789 - accuracy: 0.3328
Epoch 2/5
919/919 - 3s - loss: 1.3797 - accuracy: 0.4357
Epoch 3/5
919/919 - 3s - loss: 1.2244 - accuracy: 0.5481
Epoch 4/5
919/919 - 3s - loss: 1.1033 - accuracy: 0.6329
Epoch 5/5
919/919 - 3s - loss: 0.9991 - accuracy: 0.6954


****************************************************************************************


Training time: 0:00:21.064079
test loss  0.8581321239471436
test accuracy  0.8004525899887085


## CASE 1 : Digits 5-9 is trained then last layer of network trained on Digit 0-4

**Digit 5-9   :-  Training time: 0:00:41.670260**           

test loss  1.097296118736267

test accuracy  0.8335733413696289

**Digit 0-4   :-  Training time: 0:00:21.074518**

test loss  0.787869930267334

test accuracy  0.8914185762405396

## CASE 2 : Digits 0-4 is trained then last layer of network trained on Digit 5-9

**Digit 0-4 Training time: 0:00:22.095582**

test loss  0.638266921043396

test accuracy  0.9340338706970215


**Digit 5-9 Training time: 0:00:21.064079**

test loss  0.8581321239471436

test accuracy  0.8004525899887085